🔗 [Back to Table of Contents](https://github.com/najaeda/najaeda-tutorials#-table-of-contents)
# Chapter 2: Opening a Design with Primitives Described in Liberty Format

In this chapter, we’ll explore how to work with a [design](https://github.com/chipsalliance/rocket-chip) generated using the **Rocket Chip** generator.

While detailed information about the design can be found in [this technical report](https://www2.eecs.berkeley.edu/Pubs/TechRpts/2016/EECS-2016-17.html), our goal here is not to study its functionality.  
Instead, we’ll focus on how to load a synthesized design that makes use of a standard-cell library defined in the **Liberty format**.

The **Liberty format** is an industry-standard way to describe the behavior, timing, and characteristics of library cells in a specific technology node.

For this tutorial, we’ll use the open-source [Nangate](https://github.com/oscc-ip/nangate) library as our standard-cell library.

---

Let's begin by setting up the environment.

In [ ]:
!wget https://github.com/najaeda/najaeda-tutorials/archive/refs/heads/main.zip
!unzip main.zip
!mv najaeda-tutorials-main/benchmarks/verilog/tinyrocket/tinyrocket.v tinyrocket.v
!mv najaeda-tutorials-main/benchmarks/liberty liberty
!pip install najaeda

## Loading the liberty library and the design

After importing **najaeda** package, we construct a list of the needed **liberty** files, we load them and then we load the verilog design. 

In [ ]:
from najaeda import netlist

# Load the liberty library elements and the design
liberty_files = [
    'liberty/NangateOpenCellLibrary_typical.lib',
    'liberty/fakeram45_64x32.lib'
]
netlist.load_liberty(liberty_files)
top = netlist.load_verilog('tinyrocket.v')
print(f"Design {top.get_name()} loaded")

## Exploring the Design: More Advanced Methods

Now that the design is loaded, we can begin writing some utility functions to explore it in more depth.

In [ ]:
def explore_design(instance):
    print(f"{instance.get_name()} with model: {instance.get_model_name()}")
    for ins in instance.get_child_instances():
        explore_design(ins)

explore_design(top)

An alternative way to perform this exploration is by using the `instance_visitor` class, which offers similar functionality and flexibility.

In [ ]:
from najaeda import instance_visitor

def print_instance(instance):
    print(f"{instance.get_name()} with model: {instance.get_model_name()}")

visitor_config = instance_visitor.VisitorConfig(callback=print_instance)
instance_visitor.visit(top, visitor_config)

We can modify the `print_instance` method to print only non-primitive instances, effectively showing only the user-defined hierarchy.

In [ ]:
def print_non_primitive(instance):
    if not instance.is_primitive():
        print(f"{instance} with model: {instance.get_model_name()}")

visitor_config = instance_visitor.VisitorConfig(callback=print_non_primitive)
instance_visitor.visit(top, visitor_config)

It's also possible to pass additional arguments to the callback function used with the `instance_visitor` class.

As an example, let's implement a function that counts all primitive and non primitive instances across the entire design.  
To do this, we'll use a Python dictionary to pass a mutable counter to the callback.

In [ ]:
stats = {"primitives": 0, "non_primitives": 0}
def count_instances(instance, stats):
    if instance.is_primitive():
        stats["primitives"] += 1
    else:
        stats["non_primitives"] += 1

visitor_config = instance_visitor.VisitorConfig(callback=count_instances, args=(stats,))
instance_visitor.visit(top, visitor_config)
print(f"In design {top.get_name()}:")
print(f"Total number of primitive instances: {stats['primitives']}")
print(f"Total number of non-primitive instances: {stats['non_primitives']}")

## Extracting Statistics and Using Pandas

With `najaeda`, it’s straightforward to extract design data and prepare it for analysis in popular frameworks such as [pandas](https://pandas.pydata.org/).

In fact, `najaeda` already includes built-in methods to generate this data in a format that’s ready for use.

As an example, let’s export statistics for each instance under `top` in `JSON` format.

In [ ]:
from najaeda import stats

with open('top_stats.json', "w") as f:
  stats.dump_instance_stats_json(top, f)

Next, we’ll load the file into `Pandas` and display its contents.

In [ ]:
import pandas as pd

df = pd.read_json('top_stats.json')
pandas_data = pd.DataFrame(df.set_index("Name"))
plot = pandas_data.plot.bar(y=["terms", "nets", "instances"], stacked=True)

# Customize plot
plot.set_title("Design Statistics", fontsize=16, fontweight="bold")
plot.set_xlabel("Design Name", fontsize=12)
plot.set_ylabel("Count", fontsize=12)

plot_figure = plot.get_figure()
plot_figure.tight_layout()
plot_figure.savefig('design_stats.png')